In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
tf.config.experimental_run_functions_eagerly(True)

2024-12-20 20:55:49.556608: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 20:55:49.613136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-20 20:55:49.643083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-20 20:55:49.649798: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 20:55:49.669553: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [ ]:
# Load the data from L1_8000rpm_data/ folder
X_train = pd.read_csv('8000rpm_data_z-score/train_sample.csv')
X_test = pd.read_csv('8000rpm_data_z-score/test_sample.csv')
y_train = pd.read_csv('8000rpm_data_z-score/train_label.csv')
y_test = pd.read_csv('8000rpm_data_z-score/test_label.csv')

In [ ]:
# 构建 CNN 模型
model = keras.Sequential([
    keras.layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(1024, 1)),  # 卷积层
    keras.layers.MaxPooling1D(pool_size=2),  # 最大池化层
    keras.layers.Conv1D(64, kernel_size=3, activation='relu'),  # 卷积层
    keras.layers.MaxPooling1D(pool_size=2),  # 最大池化层
    # LSTM 层
    # tf.keras.layers.LSTM(50, return_sequences=True),
    keras.layers.Flatten(),  # 展平层
    keras.layers.Dense(128, activation='relu'),  # 全连接层
    keras.layers.Dropout(0.3),  # Dropout 防止过拟合
    keras.layers.Dense(9, activation='softmax')  # 输出层，9 类 one-hot
])

# 编译模型
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# 模型结构概览
model.summary()

In [ ]:
# 训练模型,且记录model.history.history
history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_data=(X_test, y_test))
#model.fit(X_train, y_train, epochs=60, batch_size=100, validation_split=0.2, verbose=2)

# 模型预测
y_pred = model.predict(X_test)

y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

In [ ]:
# 可视化训练过程的 loss 和 accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='train loss')
#plt.plot(history.history['val_loss'], label='validation loss')
plt.legend()
plt.title('Loss')
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='train accuracy')
#plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.legend()
plt.title('Accuracy')
plt.show()

In [ ]:
# 计算混淆矩阵
conf_mat = confusion_matrix(y_test_classes, y_pred_classes)

# 绘制混淆矩阵
plt.matshow(conf_mat, cmap='Blues')
for i in range(conf_mat.shape[0]):
    for j in range(conf_mat.shape[1]):
        plt.text(x=j, y=i, s=conf_mat[i, j], va='center', ha='center')

plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# 评估
print("Classification Report:")
print(classification_report(y_pred=y_pred_classes, y_true=y_test_classes))

In [ ]:
# 绘制ROC曲线
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
import matplotlib
import matplotlib.pyplot as plt

# 将标签二值化
y_test_bin = label_binarize(y_test_classes, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8])
y_pred_bin = label_binarize(y_pred_classes, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8])

# 计算每一类的ROC
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(9):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# 绘制ROC曲线
plt.figure()
lw = 2
colors = ['aqua', 'darkorange', 'cornflowerblue', 'red', 'green', 'yellow', 'black', 'blue', 'purple']
for i, color in zip(range(9), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
                   ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()